In [ ]:
from abc import ABC, abstractmethod
from collections import Counter, defaultdict
from dataclasses import dataclass
from enum import Enum
import itertools
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import random
from typing import Dict, List, Optional, Set, Tuple, Union

In [ ]:
from basic_utils import (
    ALL_ROLL_TUPLES,
    Box,
    BoxCategories,
    RollAction,
    RollValues,
    ScoreAction,
    ScoreCard,
    roll_first,
    remove_dice,
    roll_again,
    GameState,
    ROLL_TUPLES_BY_BOX,
)
from agents import Agent, EpsilonGreedyAgent, GreedyAgent, RandomAgent
from expected_score_utils import all_expected_scores_table, expected_scores_table, best_roll_action_for_box_with_score, best_action_by_box_with_score, greedy_best_action

A state in the game consists of a `ScoreCard` (score card state), `RollValues` (values of dice showing on the table), and `rolls_completed` from 1 to 3 within the turn. The `ScoreCard` contains all needed information from the previous turns. The `RollValues` just contains the values of the five dice that have been rolled at a given point.

`GameState` contains all three of these objects. It provides `possible_score_actions`, which gives the scores possible with a given set of dice values and score card state. I frame these as actions because at any time the player can choose to end their turn and score with one of these values. For convenience, they are sorted in descending order by score. `GameState` also provides `possible_actions`, which includes roll actions in addition to the `possible_score_actions`. The `re_roll` method takes dice that are specified by value and rolls again. Finally, `GameState` provides an `update_score` method, which updates the scorecard given a choice of box.

The current score can be accessed at any time by calling the `score` method of the `GameState`'s `scorecard`.

In [ ]:
class GreedyExpectedScoresAgentUpperBoxWeighted(Agent):

    def __init__(self, upper_box_multiplier: float, narrate=False):
        if upper_box_multiplier < 1:
            raise ValueError("upper_box_multiplier must be >= 1.")
        self.upper_box_multiplier = upper_box_multiplier
        super().__init__(narrate=narrate)

    def choose_action(self, game_state: GameState) -> Union[RollAction, ScoreAction]:
        if game_state.rolls_completed < 3:
            return greedy_best_action(game_state.roll_values, game_state.scorecard.unused_boxes, self.upper_box_multiplier)
        else:
            return game_state.sorted_possible_score_actions[0]

In [ ]:
GreedyExpectedScoresAgentUpperBoxWeighted(upper_box_multiplier=1.15, narrate=True).play_single_game()